<a href="https://colab.research.google.com/github/kolessovva/python-ai-Albina-Kolesova/blob/main/notebooks/week2b_read_csv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🌌 Week 2: Data Analysis — Чтение и проверка астрономических данных ----

**Цель**: Научиться читать CSV-файлы из репозитория GitHub в Google Colab и выполнять базовую проверку данных о звёздах и созвездиях с помощью pandas.

**Данные:**
- `stars_constellation_starTypeLabel.csv` — информация о звёздах: название, созвездие, тип звезды (например, красный сверхгигант)
- `constellations_area_abbreviation_neighborConstellationsLabel.csv` — характеристики созвездий: площадь на небе, аббревиатура, соседние созвездия

**Что мы делаем:**
1. Клонируем ваш репозиторий GitHub в Colab
2. Читаем CSV-файлы в pandas DataFrame
3. Очищаем и анализируем структуру астрономических данных
4. Проверяем целостность данных (отсутствие пропусков, корректность связей)

## 🐱 [1] Клонируем репозиторий курса в Colab

In [1]:
# 🌠 Шаг 1. Клонируем ваш репозиторий в Colab

import os

repo_name = "python-ai-Albina-Kolesova"

if not os.path.exists(repo_name):
    !git clone -q https://github.com/kolessovva/python-ai-Albina-Kolesova.git

%cd {repo_name}

print(f"✅ Репозиторий готов, теперь мы работаем внутри папки {repo_name}")
print(f"📁 Содержимое папки data:")
!ls data/

/content/python-ai-Albina-Kolesova
✅ Репозиторий готов, теперь мы работаем внутри папки python-ai-Albina-Kolesova
📁 Содержимое папки data:
constellations_area_abbreviation_neighborConstellationsLabel.csv
constellations_area_abbreviation_neighborConstellationsLabel.sparql
examples
README.md
stars_constellation_starTypeLabel.csv
stars_constellation_starTypeLabel.sparql


## 📥 [2A] Простое чтение CSV-файлов в pandas

Сначала просто прочитаем оба CSV-файла в объекты `DataFrame`, без каких‑либо изменений.

После этого мы узнаем, сколько строк загружено в каждый датасет.

In [2]:
# 🐱 Шаг 2A. Чтение CSV-файлов в pandas

import pandas as pd

stars_df = pd.read_csv("data/stars_constellation_starTypeLabel.csv")
constellations_df = pd.read_csv("data/constellations_area_abbreviation_neighborConstellationsLabel.csv")

print("✅ Загружено строк в stars_df:", len(stars_df))
print("✅ Загружено строк в constellations_df:", len(constellations_df))

✅ Загружено строк в stars_df: 2078
✅ Загружено строк в constellations_df: 527


## 🌠 [2B] Очистка и переименование столбцов

В исходных CSV-файлах есть **технические столбцы Wikidata**, которые мешают анализу астрономических данных:

- Столбцы `star`, `constellation` содержат URL-ссылки на объекты в Викиданных — для анализа достаточно читаемых названий.
- Столбцы `*Label` (`starLabel`, `constellationLabel`, `starTypeLabel`, `neighborConstellationsLabel`) содержат человекочитаемые значения.

В этом шаге мы:
- удалим столбцы с URL (`star`, `constellation`);
- переименуем читаемые столбцы для удобства:
  - `starLabel → star`
  - `constellationLabel → constellation`
  - `starTypeLabel → star_type`
  - `neighborConstellationsLabel → neighbors`
- приведём числовой столбец `area` (площадь созвездия) к типу `float`.

При приведении к числам используем:
- `pd.to_numeric(..., errors="coerce")` — преобразует значения в числа, некорректные значения превращает в `NaN`;
- `fillna(0)` — заменяет пропущенные значения (`NaN`) на 0.

> ⚠️ **Важно:** очистка URL-столбцов и унификация имён критичны для корректного объединения таблиц (например, связь звёзд с их созвездиями по названию).

In [3]:
# 🌠 Шаг 2B. Очистка и переименование столбцов

# 1) stars_df: данные о звёздах
stars_df = stars_df.drop(columns=["star", "constellation"])  # Удаляем URL Wikidata
stars_df = stars_df.rename(columns={
    "starLabel": "star",
    "constellationLabel": "constellation",
    "starTypeLabel": "star_type"
})

# 2) constellations_df: данные о созвездиях
constellations_df = constellations_df.drop(columns=["constellation"])  # Удаляем URL
constellations_df = constellations_df.rename(columns={
    "constellationLabel": "constellation",
    "neighborConstellationsLabel": "neighbors"
})

# Приводим площадь созвездия к числовому типу
constellations_df["area"] = pd.to_numeric(
    constellations_df["area"], errors="coerce"
).fillna(0).astype(float)

print("✅ Данные очищены:")
print(f"   • Звёзды: {stars_df.shape[0]} записей, столбцы: {list(stars_df.columns)}")
print(f"   • Созвездия: {constellations_df.shape[0]} записей, столбцы: {list(constellations_df.columns)}")

✅ Данные очищены:
   • Звёзды: 2078 записей, столбцы: ['star', 'constellation', 'star_type']
   • Созвездия: 527 записей, столбцы: ['constellation', 'area', 'abbreviation', 'neighbors']


## 🔭 [3] Обзор данных: структура и первые строки

Сделаем короткий обзор обоих DataFrame:

- посмотрим размер таблицы (`shape`);
- выведем список столбцов;
- посмотрим первые несколько строк;
- дополнительно посчитаем базовую статистику по площади созвездий (`area` в кв. градусах).

Для удобства напишем функцию `show_info(df, name)`, чтобы не повторять код дважды.

In [5]:
# 🌠 Шаг 3. Обзор данных (исправленная версия)

def show_info(df, name, n=5):
    """Краткий обзор DataFrame: имя, размер, список столбцов и первые строки."""
    print(f"\n🌌 {name}")
    print(f"Размер: {df.shape[0]} строк × {df.shape[1]} столбцов")
    print("Столбцы:", ", ".join(df.columns))
    print("\nПервые строки:")
    display(df.head(n))

# Обзор таблицы звёзд
show_info(stars_df, "Звёзды и их типы (stars_df)")

# Обзор таблицы созвездий
show_info(constellations_df, "Созвездия: площадь и соседи (constellations_df)")

# Статистика по площади созвездий
print("\n📐 Статистика по площади созвездий (кв. градусы):")
area_stats = constellations_df["area"].describe().round(2)
print(area_stats)

# Распределение типов звёзд
print("\n⭐ Распределение типов звёзд:")
print(stars_df["star_type"].value_counts())

# ✨ ИСПРАВЛЕНО: Топ-5 самых больших созвездий БЕЗ дубликатов
print("\n🏆 Топ-5 самых больших созвездий по площади (без дубликатов):")
# Убираем дубликаты по названию созвездия перед сортировкой
unique_constellations = constellations_df.drop_duplicates(subset=["constellation"])
top5 = unique_constellations.nlargest(5, "area")[["constellation", "area", "abbreviation"]]

# Добавим порядковый номер для наглядности
top5 = top5.reset_index(drop=True)
top5.index = top5.index + 1
top5.index.name = "Ранг"

display(top5)


🌌 Звёзды и их типы (stars_df)
Размер: 2078 строк × 3 столбцов
Столбцы: star, constellation, star_type

Первые строки:


,star,constellation,star_type
0,PMMR 179,Малое Магелланово Облако,красный сверхгигант
1,PMMR 94,Малое Магелланово Облако,красный сверхгигант
2,SkKM 185,Малое Магелланово Облако,красный сверхгигант
3,PMMR 48,Малое Магелланово Облако,красный сверхгигант
4,PMMR 47,Малое Магелланово Облако,красный сверхгигант



🌌 Созвездия: площадь и соседи (constellations_df)
Размер: 527 строк × 4 столбцов
Столбцы: constellation, area, abbreviation, neighbors

Первые строки:


,constellation,area,abbreviation,neighbors
0,Волопас,907.0,Boo,Дракон
1,Волопас,907.0,Boo,Дева
2,Волопас,907.0,Boo,Змея
3,Волопас,907.0,Boo,Большая Медведица
4,Волопас,907.0,Boo,Волосы Вероники



📐 Статистика по площади созвездий (кв. градусы):
count     527.00
mean      537.22
std       374.01
min         0.00
25%       237.00
50%       469.00
75%       804.00
max      1303.00
Name: area, dtype: float64

⭐ Распределение типов звёзд:
star_type
переменная звезда      1791
красный сверхгигант     210
голубой сверхгигант      77
Name: count, dtype: int64

🏆 Топ-5 самых больших созвездий по площади (без дубликатов):


,constellation,area,abbreviation
Ранг,,,
1,Гидра,1303.0,Hya
2,Дева,1294.0,Vir
3,Большая Медведица,1280.0,UMa
4,Кит,1231.0,Cet
5,Геркулес,1225.0,Her


In [8]:
# То же самое, но в миллионах долларов (млн $)
# stats_millions = (df_cost["capital_cost"] / 1e6).describe().round(2)
# print("\n📈 Статистика по бюджету в млн $:")
# print(stats_millions)


📊 Жанры, страны и продолжительность (df_genre)
Размер: (2596, 5)
Столбцы: film, genre, country, duration, capital_cost

Первые строки:
                        film                                   genre  \
0                Мэри и Макс                       взрослая анимация   
1  Отважный маленький тостер  экранизация литературного произведения   
2                  Ренессанс                               киберпанк   
3                  Ренессанс                                 неонуар   
4     Приключения кота Фрица  экранизация литературного произведения   

      country  duration  capital_cost  
0   Австралия        90     8240000.0  
1         США        90     2300000.0  
2  Люксембург       101    18000000.0  
3     Франция       101    18000000.0  
4         США        75      700000.0  

📊 Год выпуска, оценки и рейтинги (df_assessment)
Размер: (10383, 5)
Столбцы: film, publicationYear, assessment, outcome, reviewScore

Первые строки:
                             film  public

## ✅ [4] Быстрая проверка и валидация данных

Здесь мы посмотрим:

- сколько **уникальных** звёзд и созвездий есть в данных;
- **какие созвездия содержат больше всего звёзд** (Топ‑5);
- **какие типы звёзд встречаются чаще всего** (Топ‑10);
- **сколько соседей у созвездий** (проверка полноты данных).

Функция `value_counts()`:
- считает, сколько раз каждое значение встречается в столбце;
- сортирует результаты по убыванию.

Метод `.head()` берёт первые N строк, поэтому  
`stars_df["constellation"].value_counts().head()` даёт **Топ‑5 созвездий по числу звёзд**.

In [6]:
# ✅ Шаг 4. Быстрая проверка и валидация данных

print("🔭 Валидация астрономических данных\n")

# === Анализ звёзд ===
print("🌟 Звёзды (stars_df):")
print(f"  • Уникальных звёзд: {stars_df['star'].nunique()}")
print(f"  • Уникальных созвездий: {stars_df['constellation'].nunique()}")
print(f"  • Уникальных типов звёзд: {stars_df['star_type'].nunique()}")

print("\nТоп-5 созвездий по количеству звёзд:")
top_constellations = stars_df["constellation"].value_counts().head()
for i, (const, count) in enumerate(top_constellations.items(), 1):
    print(f"  {i}. {const}: {count} звёзд")

print("\nТоп-10 типов звёзд:")
top_types = stars_df["star_type"].value_counts().head(10)
for i, (stype, count) in enumerate(top_types.items(), 1):
    print(f"  {i}. {stype}: {count} звёзд")

# === Анализ созвездий ===
print("\n🌌 Созвездия (constellations_df):")
# Уникальные созвездия (без дубликатов из-за соседей)
unique_cons = constellations_df.drop_duplicates(subset=["constellation"])
print(f"  • Уникальных созвездий: {unique_cons['constellation'].nunique()}")
print(f"  • Диапазон площадей: {unique_cons['area'].min():.1f} — {unique_cons['area'].max():.1f} кв. градусов")

# Подсчёт количества соседей на созвездие
neighbors_count = constellations_df.groupby("constellation").size()
print(f"  • Среднее число соседей на созвездие: {neighbors_count.mean():.1f}")
print(f"  • Мин/макс соседей: {neighbors_count.min()} — {neighbors_count.max()}")

print("\nТоп-5 созвездий по числу соседей:")
top_neighbors = neighbors_count.nlargest(5)
for i, (const, count) in enumerate(top_neighbors.items(), 1):
    abbr = constellations_df[constellations_df["constellation"] == const].iloc[0]["abbreviation"]
    print(f"  {i}. {const} ({abbr}): {count} соседей")

# === Кросс-валидация ===
print("\n🔍 Кросс-проверка согласованности данных:")
stars_cons = set(stars_df["constellation"].unique())
const_cons = set(unique_cons["constellation"].unique())
only_in_stars = stars_cons - const_cons
only_in_const = const_cons - stars_cons

print(f"  • Созвездий в таблице звёзд: {len(stars_cons)}")
print(f"  • Созвездий в таблице созвездий: {len(const_cons)}")
if only_in_stars:
    print(f"  ⚠️  Созвездия только в таблице звёзд: {', '.join(sorted(only_in_stars))[:50]}...")
if only_in_const:
    print(f"  ⚠️  Созвездия только в таблице созвездий: {', '.join(sorted(only_in_const))[:50]}...")
if not only_in_stars and not only_in_const:
    print("  ✅ Все созвездия согласованы между таблицами!")

🔭 Валидация астрономических данных

🌟 Звёзды (stars_df):
  • Уникальных звёзд: 2021
  • Уникальных созвездий: 351
  • Уникальных типов звёзд: 3

Топ-5 созвездий по количеству звёзд:
  1. Messier 37: 286 звёзд
  2. Messier 67: 118 звёзд
  3. галактика Треугольника: 112 звёзд
  4. омега Центавра: 81 звёзд
  5. NGC 884: 77 звёзд

Топ-10 типов звёзд:
  1. переменная звезда: 1791 звёзд
  2. красный сверхгигант: 210 звёзд
  3. голубой сверхгигант: 77 звёзд

🌌 Созвездия (constellations_df):
  • Уникальных созвездий: 97
  • Диапазон площадей: 0.0 — 1303.0 кв. градусов
  • Среднее число соседей на созвездие: 5.4
  • Мин/макс соседей: 1 — 14

Топ-5 созвездий по числу соседей:
  1. Гидра (Hya): 14 соседей
  2. Стрелец (Sgr): 10 соседей
  3. Эридан (Eri): 10 соседей
  4. Водолей (Aqr): 9 соседей
  5. Геркулес (Her): 9 соседей

🔍 Кросс-проверка согласованности данных:
  • Созвездий в таблице звёзд: 351
  • Созвездий в таблице созвездий: 97
  ⚠️  Созвездия только в таблице звёзд: ** BID 1, ** DUN 11

## 📝 Summary

**Что мы сделали в этом ноутбуке (Week 2):**

- ✅ Клонировали репозиторий GitHub в Colab
- ✅ Прочитали 2 CSV-файла из `data/examples/`
- ✅ Удалили URL Wikidata и переименовали столбцы (`*Label → короткие имена`)
- ✅ Проверили структуру данных (размер, столбцы, первые строки)
- ✅ Выполнили быструю валидацию:
  - количество уникальных фильмов, стран, жанров
  - диапазоны значений
  - топ стран и жанров по числу записей
  - типы оценок и результатов

Теперь у нас есть **аккуратные, проверенные таблицы**, с которыми удобно работать дальше.

В отдельном ноутбуке для следующей недели мы будем использовать **те же данные** для:
- более сложного анализа (группировки, фильтрация),
- и построения визуализаций (графики и диаграммы). 🎨